In [ ]:
from Extract_ephys_from_struct import ExtractEphysData

#now test the class
#initialize the class
EED = ExtractEphysData('/Users/cresp1el/Documents/MATLAB/lmc_20ms_data.mat')

#test the methods of the class
EED.load_matfiles_printdata()

#test the extract_ephys_data method
EED.extract_ephys_data('Lmc_opsin', 'lmc_ch_1_3094_rec1', 'cid134')

#what are the group names?
EED.group_names